## Working with QTM in Vgrid DGGS

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.vgrid.vn/lab/index.html?path=vgrid/10_qtm.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/main/docs/notebooks/10_qtm.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD?filepath=docs/notebooks/10_qtm.ipynb)
[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeoshub/vgrid/blob/main/docs/notebooks/10_qtm.ipynb)
[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/10_qtm.ipynb)

Full Vgrid DGGS documentation is available at [vgrid document](https://vgrid.gishub.vn).

To work with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/). Full Vgridpandas DGGS documentation is available at [vgridpandas document](https://vgridpandas.gishub.vn).

To work with Vgrid DGGS in QGIS, install the [Vgrid Plugin](https://plugins.qgis.org/plugins/vgridtools/).

To visualize DGGS in Maplibre GL JS, try the [vgrid-maplibre](https://www.npmjs.com/package/vgrid-maplibre) library.

For an interactive demo, visit the [Vgrid Homepage](https://vgrid.vn).

### Install vgrid
Uncomment the following line to install [vgrid](https://pypi.org/project/vgrid/).

In [ ]:
# %pip install vgrid --upgrade

### latlon2qtm

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2qtm

lat = 10.775276
lon = 106.706797
res = 14
qtm_id = latlon2qtm(lat, lon, res)
qtm_id

### QTM to Polygon

In [ ]:
from vgrid.conversion.dggs2geo.qtm2geo import qtm2geo

qtm_geo = qtm2geo(qtm_id)
qtm_geo

### QTM to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.qtm2geo import qtm2geojson

qtm_geojson = qtm2geojson(qtm_id)
qtm_geojson

### Vector to QTM

In [ ]:
from vgrid.conversion.vector2dggs.vector2qtm import vector2qtm

file_path = "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/polygon2.geojson"
vector_to_qtm = vector2qtm(
    file_path,
    resolution=17,
    compact=False,
    topology=False,
    predicate="intersects",
    output_format="gpd",
)
# Visualize the output
vector_to_qtm.plot(edgecolor="white")

### QTM Compact

In [ ]:
from vgrid.conversion.dggscompact import qtmcompact

qtm_compacted = qtmcompact(vector_to_qtm, qtm_id="qtm", output_format="gpd")
qtm_compacted.plot(edgecolor="white")

### QTM Expand

In [ ]:
from vgrid.conversion.dggscompact import qtmexpand

qtm_expanded = qtmexpand(
    vector_to_qtm, resolution=18, qtm_id="qtm", output_format="gpd"
)
qtm_expanded.plot(edgecolor="white")

### QTM Binning

In [ ]:
from vgrid.binning.qtmbin import qtmbin

file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/csv/dist1_pois.csv"
)
stats = "count"
qtm_bin = qtmbin(
    file_path,
    resolution=16,
    stats=stats,
    # numeric_field="confidence",
    # category="category",
    output_format="gpd",
)
qtm_bin.plot(
    column=stats,  # numeric column to base the colors on
    cmap="Spectral_r",  # color scheme (matplotlib colormap)
    legend=True,
    linewidth=0.2,  # boundary width (optional)
)

### Raster to QTM

#### Download and open raster

In [ ]:
# %pip install folium

In [ ]:
from vgrid.utils.io import download_file
import rasterio
from rasterio.plot import show

raster_url = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/raster/rgb.tif"
)
raster_file = download_file(raster_url)
src = rasterio.open(raster_file, "r")
print(src.meta)
show(src)

#### Convert raster to QTM

In [ ]:
from vgrid.conversion.raster2dggs.raster2qtm import raster2qtm

raster_to_qtm = raster2qtm(raster_file, resolution=23, output_format="gpd")

# Visualize the output
import folium

m = folium.Map(tiles="CartoDB positron", max_zoom=28)

qtm_layer = folium.GeoJson(
    raster_to_qtm,
    style_function=lambda x: {
        "fillColor": f"rgb({x['properties']['band_1']}, {x['properties']['band_2']}, {x['properties']['band_3']})",
        "fillOpacity": 1,
        "color": "black",
        "weight": 1,
    },
    popup=folium.GeoJsonPopup(
        fields=["qtm", "band_1", "band_2", "band_3"],
        aliases=["QTM ID", "Band 1", "Band 2", "Band 3"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(qtm_layer.get_bounds())

# Display the map
m

### QTM Generator

In [ ]:
from vgrid.generator.qtmgrid import qtmgrid

qtm_grid = qtmgrid(resolution=3, output_format="gpd")
# qtm_grid = qtmgrid(resolution=18,bbox=[106.699007, 10.762811, 106.717674, 10.778649],output_format="gpd")
qtm_grid.plot(edgecolor="white")
qtm_grid.to_file("qtm.geojson")

### QTM Inspect

In [ ]:
from vgrid.stats.qtmstats import qtminspect

resolution = 6
qtm_inspect = qtminspect(resolution)
qtm_inspect.head()

### QTM Normalized Area Histogram

In [ ]:
from vgrid.stats.qtmstats import qtm_norm_area_hist

qtm_norm_area_hist(qtm_inspect)

### Distribution of QTM Area Distortions

In [ ]:
from vgrid.stats.qtmstats import qtm_norm_area

qtm_norm_area(qtm_inspect)

### QTM IPQ Compactness Histogram

Isoperimetric Inequality (IPQ) Compactness (suggested by Osserman, 1987):
$$C_{IPQ} = \frac{4 \pi A}{p^2}$$
The range of the IPQ compactness metric is [0,1]. 

A circle represents the maximum compactness with a value of 1. 

As shapes become more irregular or elongated, their compactness decreases toward 0.

In [ ]:
from vgrid.stats.qtmstats import qtm_compactness_hist

qtm_compactness_hist(qtm_inspect)

### Distribution of QTM IPQ Compactness

In [ ]:
from vgrid.stats.qtmstats import qtm_compactness

qtm_compactness(qtm_inspect)

### QTM Statistics

Characteristic Length Scale (CLS - suggested by Ralph Kahn): the diameter of a spherical cap of the same cell's area

In [ ]:
from vgrid.stats import qtmstats

qtmstats("km")